# Binance OHLCV

In [ ]:
from clients import BinanceClient
import ccxt

binance = BinanceClient()

symbol = 'BTC/USDT'
timeframe = '1h'
df = binance.get_ohlcv(symbol, timeframe)

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Candlestick(x=df['datetime'],
                open=df['open'],
                high=df['high'],
                low=df['low'],
                close=df['close'])])

fig.show()

# Binance Balance

In [ ]:
from clients import BinanceClient

binance = BinanceClient()
binance.con_trading_login()
binance.fetch_balance()

# Ethereum Mempool

In [1]:
from clients import EthClient
ec = EthClient()

True


In [2]:
import json
import time

while True:
    func_obj, results = ec.get_pending_tx_uniswap()
    if results != []:
        break
    time.sleep(3)
        # output json
with open('./data/uniswap_pending.json', 'w') as f:
    json.dump(results, f)

In [3]:
results

[{'params': {'tokenIn': '0xd8F1460044925d2D5c723C7054cd9247027415B7',
   'tokenOut': '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2',
   'fee': 10000,
   'recipient': '0xf3F32E156E95c6249AA646c86855306F913bFF34',
   'deadline': 1712257062,
   'amountIn': 24999999999999997902848,
   'amountOutMinimum': 97948493064137477,
   'sqrtPriceLimitX96': 0},
  'timestamp': '2024-04-04T14:48:00.737256'}]

Price of 1 Wrapped Ether (WETH) in Tether USD (USDT): 3432.131383941539
Price of 1 Tether USD (USDT) in Wrapped Ether (WETH): 0.0002913641373634062
